
Step 1. [Download history price and clean data](/notebooks/forex/hawkeye/1_download.ipynb)

Step 2. [Use monte carlo simulate each stock's movement for 10000 times](/notebooks/forex/hawkeye/2_process.ipynb)

**Step 3. Apply some heuristic rules to give stock recommendation**

In [156]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

sys.path.append('/opt/hawkeye')
from asx import *

this_week=20190614
last_week=20190607
path = f'/opt/hawkeye/data/{this_week}/result.csv'
last_path = f'/opt/hawkeye/data/{last_week}/result.csv'

def week_return_avg(df):
    return round(df['week_return'].sum()/len(df),4)

df = pd.read_csv(path,
                 usecols=[
                     'code', 'last_date', 'start price', 'sim_mean',
                     'sim_diff', 'VaR 99%', 'VaR 99% Percent', 'volume_mean',
                     'return_mean', 'return_sigma'
                 ],
                 index_col='code')
last_df = pd.read_csv(last_path,
                      usecols=[
                          'code', 'last_date', 'start price', 'sim_mean',
                          'sim_diff', 'VaR 99%', 'VaR 99% Percent',
                          'volume_mean', 'return_mean', 'return_sigma'
                      ],
                      index_col='code')

# remove bottom volume and bottom price
volume_threshold = df['volume_mean'].quantile(0.1)
df = df[(df['volume_mean'] > volume_threshold) & (df['start price'] > 0.05)]

df['return'] = round(df['sim_diff'] / df['start price'] * 100, 3)
df['return_rank'] = round(df['return'].rank(pct=True) * 100, 3)
df['risk_rank'] = round(df['VaR 99% Percent'].rank(pct=True) * 100, 3)
df['volume_rank'] = round(df['volume_mean'].rank(pct=True) * 100, 3)

df['last_return'] = round(last_df['sim_diff'] / last_df['start price'] * 100,3)
df['return_increase'] = df['return'] - df['last_return']
df['week_return'] = round(
    (df['start price'] - last_df['start price']) / df['start price'] * 100, 3)
df.drop(columns=['sim_diff', 'sim_mean', 'VaR 99%','VaR 99% Percent', 'volume_mean'], inplace=True)

print(f'Total stock number = {len(df)}')

print('70%% pricetile = %s' % df['start price'].quantile(0.7))
print('Sim return 90%% percentile = %s' % df['return'].quantile(0.9))

print(f'Volume 90% percentile = {volume_threshold}')

print(len(df))
df.drop(index='MMJ', inplace=True)
df.sort_values(by='return', ascending=False).head(20)

Total stock number = 999
70% pricetile = 2.0119999999999982
Sim return 90% percentile = 1.0826000000000002
Volume 90% percentile = 14751.404040404042
999


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
FTC,2019-06-14,0.155,0.025928,0.190769,5.079,99.900,99.700,73.574,4.869,0.210,6.452
UWL,2019-06-14,1.310,0.024609,0.075059,4.815,99.800,82.382,79.780,NaN,NaN,NaN
LTR,2019-06-14,0.115,0.019955,0.096789,3.860,99.700,96.797,99.499,3.855,0.005,8.696
FEX,2019-06-14,0.110,0.019293,0.089630,3.848,99.600,94.695,98.899,4.226,-0.378,-4.545
8CO,2019-06-14,0.145,0.019064,0.076112,3.803,99.499,85.986,50.450,3.857,-0.054,-10.345
NET,2019-06-14,0.110,0.019104,0.086727,3.732,99.399,92.492,91.091,2.954,0.778,12.727
ANO,2019-06-14,5.920,0.018382,0.068696,3.653,99.299,80.480,28.328,3.785,-0.132,-9.459
TNY,2019-06-14,1.190,0.018666,0.109975,3.639,99.199,99.099,31.431,NaN,NaN,NaN
ISX,2019-06-14,0.635,0.016686,0.056640,3.315,99.099,67.968,87.487,3.232,0.083,-2.362


In [157]:
# high mean return 
high_return=df.sort_values(by='return', ascending=False).head(10)
print('High return last week performance avg. = %s%%'% week_return_avg(high_return))
high_return

High return last week performance avg. = -0.2682%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
FTC,2019-06-14,0.155,0.025928,0.190769,5.079,99.900,99.700,73.574,4.869,0.210,6.452
UWL,2019-06-14,1.310,0.024609,0.075059,4.815,99.800,82.382,79.780,NaN,NaN,NaN
LTR,2019-06-14,0.115,0.019955,0.096789,3.860,99.700,96.797,99.499,3.855,0.005,8.696
FEX,2019-06-14,0.110,0.019293,0.089630,3.848,99.600,94.695,98.899,4.226,-0.378,-4.545
8CO,2019-06-14,0.145,0.019064,0.076112,3.803,99.499,85.986,50.450,3.857,-0.054,-10.345
NET,2019-06-14,0.110,0.019104,0.086727,3.732,99.399,92.492,91.091,2.954,0.778,12.727
ANO,2019-06-14,5.920,0.018382,0.068696,3.653,99.299,80.480,28.328,3.785,-0.132,-9.459
TNY,2019-06-14,1.190,0.018666,0.109975,3.639,99.199,99.099,31.431,NaN,NaN,NaN
ISX,2019-06-14,0.635,0.016686,0.056640,3.315,99.099,67.968,87.487,3.232,0.083,-2.362


In [158]:
# high price

high_price=df[df['start price']>1].sort_values(by='return', ascending=False).head(10)
print('High price last week performance avg. = %s%%'% week_return_avg(high_price))
high_price

High price last week performance avg. = 4.4814%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
UWL,2019-06-14,1.310,0.024609,0.075059,4.815,99.800,82.382,79.780,NaN,NaN,NaN
ANO,2019-06-14,5.920,0.018382,0.068696,3.653,99.299,80.480,28.328,3.785,-0.132,-9.459
TNY,2019-06-14,1.190,0.018666,0.109975,3.639,99.199,99.099,31.431,NaN,NaN,NaN
DUB,2019-06-14,1.420,0.014615,0.059723,2.810,98.699,76.176,70.771,2.318,0.492,21.127
CIA,2019-06-14,3.240,0.011548,0.039920,2.274,97.798,48.148,55.856,1.960,0.314,9.568
BUB,2019-06-14,1.075,0.010646,0.056375,2.108,97.097,72.873,96.196,2.099,0.009,-0.465
Z1P,2019-06-14,2.950,0.010513,0.046225,2.045,96.997,57.257,89.389,2.310,-0.265,-2.712
5GN,2019-06-14,1.420,0.010393,0.046456,1.987,96.797,59.560,61.361,1.720,0.267,17.606
CRD,2019-06-14,1.740,0.009947,0.045742,1.960,96.697,57.658,52.452,1.866,0.094,1.149


In [159]:
# highest price

highest_price=df[df['return']>0.8].sort_values(by='start price', ascending=False).head(10)
print('Highest price last week performance avg. = %s%%'% week_return_avg(highest_price))
highest_price

Highest price last week performance avg. = 3.8538%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
MFG,2019-06-14,47.60,0.005717,0.016877,1.099,90.390,9.309,67.467,1.083,0.016,3.718
CUV,2019-06-14,34.10,0.005243,0.036112,1.036,89.489,49.650,35.235,0.771,0.265,7.067
APX,2019-06-14,25.82,0.005689,0.033277,1.100,90.490,45.145,73.674,1.245,-0.145,-3.447
JIN,2019-06-14,18.75,0.009431,0.029942,1.872,96.496,30.631,55.656,1.692,0.180,8.000
IEL,2019-06-14,18.03,0.005005,0.026398,0.960,88.488,31.732,71.071,0.964,-0.004,7.321
BRG,2019-06-14,16.97,0.005061,0.023582,0.992,88.889,25.425,55.455,0.933,0.059,1.414
CHC,2019-06-14,11.52,0.004294,0.011989,0.826,85.185,2.653,83.483,0.743,0.083,5.035
LOV,2019-06-14,11.28,0.005302,0.032048,1.069,89.890,44.144,52.853,1.223,-0.154,-0.355
APE,2019-06-14,9.90,0.005588,0.022739,1.056,89.790,21.622,14.615,1.052,0.004,-1.010


In [163]:
# low risk
low_risk = df.sort_values(by='risk_rank', ascending=True).head(20).sort_values(by='return', ascending=False)
print('Low risk last week performance avg. = %s%%'% week_return_avg(low_risk))
low_risk

Low risk last week performance avg. = 1.48%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
ASX,2019-06-14,81.000,0.002568,0.009643,0.496,73.524,1.902,57.257,0.495,0.001,2.111
TCL,2019-06-14,14.510,0.002049,0.008477,0.406,67.568,1.502,95.495,0.380,0.026,1.723
SHL,2019-06-14,27.500,0.002129,0.008969,0.405,67.367,1.602,76.877,0.411,-0.006,3.709
CMA,2019-06-14,2.830,0.001996,0.007329,0.385,66.066,0.601,67.067,0.337,0.048,3.180
AST,2019-06-14,1.945,0.001964,0.008382,0.384,65.916,1.201,94.595,0.342,0.042,4.627
CMW,2019-06-14,1.200,0.001892,0.008086,0.371,65.365,0.801,93.293,0.356,0.015,0.833
AMC,2019-06-14,15.650,0.001951,0.009398,0.363,64.815,2.002,93.794,0.402,-0.039,-0.767
GOZ,2019-06-14,4.320,0.001708,0.007992,0.331,62.813,1.001,68.168,0.301,0.030,0.694
CIP,2019-06-14,3.210,0.001605,0.008074,0.316,61.712,1.301,41.041,0.307,0.009,0.623


In [162]:
# high volume
high_volume = df.sort_values(by='volume_rank', ascending=False).head(20).sort_values(by='return', ascending=False)
print('Highest volume last week performance avg. = %s%%'% week_return_avg(high_volume))
high_volume

Highest volume last week performance avg. = 2.1966%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
LTR,2019-06-14,0.115,0.019955,0.096789,3.860,99.700,96.797,99.499,3.855,0.005,8.696
FEX,2019-06-14,0.110,0.019293,0.089630,3.848,99.600,94.695,98.899,4.226,-0.378,-4.545
AVH,2019-06-14,0.360,0.013883,0.072828,2.689,98.498,89.089,98.498,2.774,-0.085,5.556
FMG,2019-06-14,8.800,0.007762,0.029624,1.586,94.895,33.133,99.900,1.370,0.216,10.795
CLA,2019-06-14,0.064,0.007115,0.101932,1.109,90.591,99.199,98.198,0.357,0.752,32.813
MGR,2019-06-14,3.210,0.003611,0.011091,0.697,82.032,2.102,99.600,0.621,0.076,2.492
TLS,2019-06-14,3.870,0.003052,0.012511,0.621,79.479,5.005,100.000,0.521,0.100,3.876
NEC,2019-06-14,2.000,0.002747,0.023184,0.572,76.777,27.127,98.799,0.592,-0.020,1.750
MPL,2019-06-14,3.380,0.002771,0.017488,0.568,76.627,15.415,98.599,0.552,0.016,0.888


In [164]:
# high return increase in last week
high_return_increase = df.sort_values(by='return_increase',
                                      ascending=False).head(10)
print('Highest return increase last week performance avg. = %s%%'% week_return_avg(high_return_increase))
high_return_increase

Highest return increase last week performance avg. = 21.7534%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
CUP,2019-06-14,0.750,0.004316,0.045827,0.815,84.685,65.766,14.214,-0.286,1.101,40.000
BKY,2019-06-14,0.385,0.000805,0.064948,0.130,48.498,90.390,45.646,-0.885,1.015,24.675
FZO,2019-06-14,0.135,0.000185,0.072066,0.143,49.750,93.594,50.851,-0.757,0.900,14.815
FMS,2019-06-14,0.052,0.006501,0.068578,1.288,92.392,88.789,46.647,0.434,0.854,28.846
NET,2019-06-14,0.110,0.019104,0.086727,3.732,99.399,92.492,91.091,2.954,0.778,12.727
CLA,2019-06-14,0.064,0.007115,0.101932,1.109,90.591,99.199,98.198,0.357,0.752,32.813
GSM,2019-06-14,0.080,-0.005263,0.046054,-1.038,3.604,77.477,1.602,-1.778,0.740,25.000
BSX,2019-06-14,0.096,0.001123,0.055311,0.230,56.757,82.883,37.237,-0.385,0.615,19.792
CGA,2019-06-14,0.325,0.004533,0.059873,0.952,88.238,83.684,0.100,0.353,0.599,3.077


In [165]:
# ASX 20
asx_20_list = get_asx_20_list()
asx_20=df[df.index.isin(asx_20_list)].sort_values(by='return', ascending=False)
print('ASX 20 last week performance avg. = %s%%'% week_return_avg(asx_20))
asx_20

ASX 20 last week performance avg. = 1.3436%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
TLS,2019-06-14,3.87,0.003052,0.012511,0.621,79.479,5.005,100.000,0.521,0.100,3.876
RIO,2019-06-14,105.34,0.003105,0.015542,0.583,77.477,11.311,84.484,0.479,0.104,6.968
BHP,2019-06-14,40.30,0.002273,0.012704,0.460,71.221,7.007,97.397,0.323,0.137,6.104
BXB,2019-06-14,12.97,0.002152,0.010432,0.420,68.719,3.604,93.493,0.405,0.015,1.850
TCL,2019-06-14,14.51,0.002049,0.008477,0.406,67.568,1.502,95.495,0.380,0.026,1.723
AMC,2019-06-14,15.65,0.001951,0.009398,0.363,64.815,2.002,93.794,0.402,-0.039,-0.767
WES,2019-06-14,35.66,0.001249,0.013623,0.252,58.458,11.211,88.689,0.368,-0.116,-5.580
ANZ,2019-06-14,28.22,0.001168,0.014696,0.248,58.158,12.913,96.296,0.297,-0.049,-0.532
CBA,2019-06-14,79.77,0.001248,0.011746,0.241,57.558,5.806,90.390,0.249,-0.008,-0.288


In [166]:
# highest return in ASX 200
asx_200_list = get_asx_200_list()
asx_200=df[df.index.isin(asx_200_list)].sort_values(by='return', ascending=False).head(20)
print('Top 20 of ASX 200 last week performance avg. = %s%%'% week_return_avg(asx_200))
asx_200

Top 20 of ASX 200 last week performance avg. = 3.0694%


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
FMG,2019-06-14,8.80,0.007762,0.029624,1.586,94.895,33.133,99.900,1.370,0.216,10.795
LYC,2019-06-14,2.77,0.006721,0.050607,1.332,93.093,68.368,97.598,1.186,0.146,0.000
APX,2019-06-14,25.82,0.005689,0.033277,1.100,90.490,45.145,73.674,1.245,-0.145,-3.447
MFG,2019-06-14,47.60,0.005717,0.016877,1.099,90.390,9.309,67.467,1.083,0.016,3.718
NAN,2019-06-14,4.93,0.005349,0.025646,1.096,90.290,29.930,77.477,0.793,0.303,9.939
BRG,2019-06-14,16.97,0.005061,0.023582,0.992,88.889,25.425,55.455,0.933,0.059,1.414
IEL,2019-06-14,18.03,0.005005,0.026398,0.960,88.488,31.732,71.071,0.964,-0.004,7.321
DHG,2019-06-14,3.34,0.004529,0.033834,0.883,85.986,45.946,84.184,0.842,0.041,2.994
CHC,2019-06-14,11.52,0.004294,0.011989,0.826,85.185,2.653,83.483,0.743,0.083,5.035


In [167]:
# high last week return
df.sort_values(by='week_return', ascending=False).head(20).sort_values(by='return', ascending=False)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
DUB,2019-06-14,1.420,0.014615,0.059723,2.810,98.699,76.176,70.771,2.318,0.492,21.127
5GN,2019-06-14,1.420,0.010393,0.046456,1.987,96.797,59.560,61.361,1.720,0.267,17.606
ALK,2019-06-14,0.415,0.009412,0.049031,1.814,96.196,64.364,80.180,1.377,0.437,18.072
M7T,2019-06-14,0.450,0.009076,0.041900,1.773,95.996,53.053,60.160,1.278,0.495,20.000
FMS,2019-06-14,0.052,0.006501,0.068578,1.288,92.392,88.789,46.647,0.434,0.854,28.846
SVM,2019-06-14,0.120,0.006250,0.040174,1.277,92.192,53.854,64.164,0.812,0.465,17.500
SO4,2019-06-14,0.800,0.006615,0.030287,1.250,91.992,35.836,33.333,0.838,0.412,20.000
BOT,2019-06-14,0.130,0.006368,0.042370,1.240,91.592,56.557,76.376,0.814,0.426,23.077
CLA,2019-06-14,0.064,0.007115,0.101932,1.109,90.591,99.199,98.198,0.357,0.752,32.813


## Thisk week recommendations

In [168]:
# recommendations
selection = [
    'WKT',  # high return & high risk
    'MGR',  # high price & low risk
    'MFG',  # high price & low sigma & low risk & asx 200
    'CHC',  # low sigma & low sigma & low risk & asx 200
    'TLS',  # high volume & low risk & asx 20
    'TCL',  # high volume & low risk & asx 20
    'AZJ',  # high volume & low risk
    'M7T',  # medium risk & best last week performance
]
df[df.index.isin(selection)].sort_values(by='return', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
WKT,2019-06-14,0.35,0.014051,0.051570,2.798,98.599,61.862,73.373,2.930,-0.132,-5.714
M7T,2019-06-14,0.45,0.009076,0.041900,1.773,95.996,53.053,60.160,1.278,0.495,20.000
MFG,2019-06-14,47.60,0.005717,0.016877,1.099,90.390,9.309,67.467,1.083,0.016,3.718
CHC,2019-06-14,11.52,0.004294,0.011989,0.826,85.185,2.653,83.483,0.743,0.083,5.035
MGR,2019-06-14,3.21,0.003611,0.011091,0.697,82.032,2.102,99.600,0.621,0.076,2.492
TLS,2019-06-14,3.87,0.003052,0.012511,0.621,79.479,5.005,100.000,0.521,0.100,3.876
AZJ,2019-06-14,5.34,0.002219,0.010764,0.430,69.469,3.504,98.098,0.400,0.030,2.434
TCL,2019-06-14,14.51,0.002049,0.008477,0.406,67.568,1.502,95.495,0.380,0.026,1.723


## Last week review

In [172]:
selection = [
    'FEX',  # high return & high risk
    'ANO',  # high return & high price & low sigma
    'AD8',  # high price & low sigma & low risk
    'MFG',  # high price & low sigma & low risk
    'CHC',  # low sigma & low risk & asx 200
    'JBH',  # low sigma & low risk & asx 200
    'CV1',  # best recent performance
    'FMG'
]
df[df.index.isin(selection)].sort_values(by='return', ascending=False).head(10)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
FEX,2019-06-14,0.110,0.019293,0.089630,3.848,99.600,94.695,98.899,4.226,-0.378,-4.545
ANO,2019-06-14,5.920,0.018382,0.068696,3.653,99.299,80.480,28.328,3.785,-0.132,-9.459
CV1,2019-06-14,0.145,0.010208,0.054010,1.954,96.597,70.170,53.053,2.377,-0.423,0.000
FMG,2019-06-14,8.800,0.007762,0.029624,1.586,94.895,33.133,99.900,1.370,0.216,10.795
AD8,2019-06-14,7.940,0.008233,0.026914,1.573,94.745,27.227,44.945,1.603,-0.030,2.267
MFG,2019-06-14,47.600,0.005717,0.016877,1.099,90.390,9.309,67.467,1.083,0.016,3.718
CHC,2019-06-14,11.520,0.004294,0.011989,0.826,85.185,2.653,83.483,0.743,0.083,5.035
JBH,2019-06-14,26.580,0.002600,0.014466,0.506,73.974,9.910,72.172,0.655,-0.149,-4.289


In [173]:
week_return = round(sum(df[df.index.isin(selection)]['week_return']), 3)
print('Last week return:', week_return)

Last week return: 3.522


## Signle performance review

In [175]:
def inspect(code):
    print(f'http://h.luotao.net/data/{this_week}/pic/{code}_line.png')
    print(f'http://h.luotao.net/data/{this_week}/pic/{code}.png')
    
    return df[df.index==code]

In [176]:
inspect('TNY')

http://h.luotao.net/data/20190614/pic/TNY_line.png
http://h.luotao.net/data/20190614/pic/TNY.png


,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return
code,,,,,,,,,,,
TNY,2019-06-14,1.19,0.018666,0.109975,3.639,99.199,99.099,31.431,NaN,NaN,NaN


In [177]:
df['rank']=df['return_rank']+(100-df['risk_rank'])
df.sort_values(by='rank', ascending=False).head(20)

,last_date,start price,return_mean,return_sigma,return,return_rank,risk_rank,volume_rank,last_return,return_increase,week_return,rank
code,,,,,,,,,,,,
CHC,2019-06-14,11.520,0.004294,0.011989,0.826,85.185,2.653,83.483,0.743,0.083,5.035,182.532
MFG,2019-06-14,47.600,0.005717,0.016877,1.099,90.390,9.309,67.467,1.083,0.016,3.718,181.081
MGR,2019-06-14,3.210,0.003611,0.011091,0.697,82.032,2.102,99.600,0.621,0.076,2.492,179.930
TLS,2019-06-14,3.870,0.003052,0.012511,0.621,79.479,5.005,100.000,0.521,0.100,3.876,174.474
DDR,2019-06-14,5.620,0.007496,0.024108,1.446,93.994,21.321,38.839,1.245,0.201,9.964,172.673
ASX,2019-06-14,81.000,0.002568,0.009643,0.496,73.524,1.902,57.257,0.495,0.001,2.111,171.622
MVF,2019-06-14,1.340,0.004062,0.017080,0.766,83.283,12.212,57.958,0.713,0.053,0.746,171.071
DTL,2019-06-14,2.140,0.003196,0.014170,0.613,79.079,9.009,49.750,0.644,-0.031,8.879,170.070
CKF,2019-06-14,8.350,0.003189,0.015123,0.626,79.580,9.610,47.848,0.590,0.036,2.994,169.970
